In [162]:
# Import libs
import pandas as pd
import requests
import os
import time
import random

# api stuff
api_url = ".api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5"
api_key = os.environ.get('api_key')

In [6]:
# Info
regions = ['kr', 'euw1','na1'] # Major regions of world
ranks = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER'] # All ranks
divisions = ['IV', 'III', 'II', 'I'] # All divisions

In [163]:
# Getting player names
def get_players(region, rank, division, page):
    '''
    Returns summoner names of players from specified region, rank, and division
    '''
    players_url = ("https://%s%s/%s/%s?page=%d&api_key=%s" % (region, api_url, rank, division, page, api_key))

    while True:
        resp = requests.get(players_url)
        
        if resp.status_code == 429:
            time.sleep(5)
            continue

        data = resp.json()
        return data

In [ ]:
# Loop through all pages, ranks, and divisions
df = pd.DataFrame()
for page in range(1,6):
    for rank in ranks:
        if rank in ['MASTER', 'GRANDMASTER', 'CHALLENGER']:
            if page > 1:
                continue
            else:
                names_df = pd.DataFrame(get_players('kr', rank, 'I', page))[['summonerName', 'wins', 'losses']]
                names_df['rank'] = ("%s %s" %(rank,'I'))
                df = pd.concat([df, names_df[:200]]) # get 200 players
                continue
        for division in divisions:
            print(division)
            names_df = pd.DataFrame(get_players('kr', rank, division, page))[['summonerName', 'wins', 'losses']]
            names_df['rank'] = ("%s %s" %(rank, division))
            df = pd.concat([df, names_df[:200]]) # get 200 players

print('done')

In [192]:
# Filter for active players
active_players = df[df['losses'] + df['wins'] > 100]

In [183]:
# League of Legends playerbase rank distribution as percents
rank_distribution = {
    'IRON IV': 0.03,
    'IRON III': 0.022,
    'IRON II': 0.011,
    'IRON I': 0.03,
    'BRONZE IV': 0.053,
    'BRONZE III': 0.047,
    'BRONZE II': 0.045,
    'BRONZE I': 0.036,
    'SILVER IV': 0.057,
    'SILVER III': 0.046,
    'SILVER II': 0.043,
    'SILVER I': 0.035,
    'GOLD IV': 0.064,
    'GOLD III': 0.048,
    'GOLD II': 0.048,
    'GOLD I': 0.04,
    'PLATINUM IV': 0.071,
    'PLATINUM III': 0.048,
    'PLATINUM II': 0.041,
    'PLATINUM I': 0.031,
    'EMERALD IV': 0.06,
    'EMERALD III': 0.029,
    'EMERALD II': 0.02,
    'EMERALD I': 0.012,
    'DIAMOND IV': 0.021,
    'DIAMOND III': 0.01,
    'DIAMOND II': 0.0064,
    'DIAMOND I': 0.0035,
    'MASTER I': 0.0042,
    'GRANDMASTER I': 0.0006,
    'CHALLENGER I': 0.00023
}

In [195]:
# Need to filter this data to represent playerbase
def player_distribution(distribution, data):
    '''
    Takes in players and returns a list with players proportional to distribution of ranks
    '''
    total_players = 720
    df = pd.DataFrame()
    for key, value in distribution.items():
        rank_filter = (data['rank'] == key)
        filtered_data = data[rank_filter]
        if key in ['GRANDMASTER I', 'CHALLENGER I']: # Auto include 1 player from highest ranks
            df = pd.concat([df, filtered_data['summonerName'].head(1)])
        else:
            num_players = round(value * total_players)
            df = pd.concat([df, filtered_data['summonerName'].head(num_players)])
    return df

In [ ]:
# Export to CSV
na_summonernames = player_distribution(rank_distribution, active_players).dropna()
na_summonernames = na_summonernames.rename(columns = {0: 'summonerName'})
na_summonernames.to_csv(r"C:\Users\toirt\LeagueOfLegendsPrediction\data\final_summoner_names.csv", index = False)